# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [2]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description

In this project we solve the dynamic malthusian model. The model is from the course "Economic History" here at KU and it models preindustrial malthusian growth. In this model the parameters as defined as: \
$Y$ is production \
$L$ is labor/population \
$A$ is technology \
$X$ is land, which is constant, a core mechanism \
$n$ is population growthrate 


The model is defined by the three equations below, where 1. describe how output is a function of labor technology and land, where both land and technology is constant, a core assumption. Equation 2 describes how the population growthrate is positively correlated with income, by the parameter $\eta$, another essenttial assumption. Equation 3 describes how population in the next period is given by the growthrate and the mortality rate. The model is then mean to describe how average income will be afected negatively by population growth and positively by moratality, as land and technology are set resources. 

1) $ Y_t = L_t^{1-\alpha}(AX)^\alpha $ 
2) $n_t=\eta y_t \\$
3) $L_{t-1}=n_tL_t$

## Analytical solution

While the model can be analytical solved, sympy has severe difficulties in perfoming the calculation needed. The Analytical solution to the evolution of income pr capita is hence given as: \
$y_{t+1}=\eta^{-\alpha} y_t^{1-\alpha}$ \
We then find the steady state using sympy

In [112]:
# Define symbols
y_t, y_t1, eta, alpha = sm.symbols('y_t y_{t+1} eta alpha')

# Define the equation
eq = sm.Eq(y_t1, eta**(-alpha) * y_t**(1-alpha))

# Set y_{t+1} = y_t
eq_substituted = eq.subs(y_t1, y_t)

# Solve for y_t
solution_y_t = sm.solve(eq_substituted, y_t)
solution_y_t

[(eta**(-alpha))**(1/alpha)]

As show above with great difficulty by sympy, the solution of the steady state is $y* =\frac{1}{\eta}$

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 